## Aggression SVC

In [22]:
#imports

#general imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#modeling imports
from eda_functions_1 import split_data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

#nlp imports
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [23]:
#reading in data for aggression
df = pd.read_csv('../clean data/aggression_clean_data.csv')

In [24]:
df.head()

,rev_id,comment,year,logged_in,ns,sample,split,aggression,aggression_score,label
0,37675,This is not creative Those are the dictionary...,2002,True,article,random,train,0.100000,0.000000,0
1,44816,the term standard model is itself less NPOV t...,2002,True,article,random,train,0.000000,0.111111,0
2,49851,True or false the situation as of March 2002 w...,2002,True,article,random,train,0.000000,0.100000,0
3,89320,Next maybe you could work on being less conde...,2002,True,article,random,dev,0.444444,-0.444444,0
4,93890,This page will need disambiguation,2002,True,article,random,train,0.000000,0.333333,0


In [29]:
#dropping unused columns
df.drop(columns=['rev_id','year','logged_in','ns','sample','split'],inplace=True)

In [30]:
#baseline score
df.label.value_counts(normalize=True)

0    0.852805
1    0.147195
Name: label, dtype: float64

In [31]:
#splitting data with custom function (christian wrote in script)
X_train, X_test, y_train, y_test = split_data(df,0.5)

In [32]:
#checking to see that training data is balanced 50/50
y_train.value_counts(normalize=True)

1    0.5
0    0.5
Name: label, dtype: float64

In [33]:
#checking to see that test data is not balanced 50/50 and has retained original splits
y_test.value_counts(normalize=True)

0    0.850261
1    0.149739
Name: label, dtype: float64

In [35]:
#creating pipepline for SVC and first param dict and grid search
pipe = Pipeline([
    ('cvec',CountVectorizer()),
    ('svc',SVC())
])
params = {
    'cvec__max_features':[2000,3000,4000],
    'cvec__ngram_range':[(1,1),(1,2),(2,2)],
    'cvec__stop_words':[stopwords.words('english')]
}
gs_1 = GridSearchCV(pipe,param_grid=params,cv=5)
gs_1.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('svc', SVC())]),
             param_grid={'cvec__max_features': [2000, 3000, 4000],
                         'cvec__ngram_range': [(1, 1), (1, 2), (2, 2)],
                         'cvec__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                               'our', 'ours', 'ourselves',
                                               'you', "you're", "you've",
                                               "you'll", "you'd", 'your',
                                               'yours', 'yourself',
                                               'yourselves', 'he', 'him', 'his',
                                               'himself', 'she', "she's", 'her',
                                               'hers', 'herself', 'it', "it's",
                                               'its', 'itself', ...]]})

In [37]:
#printing best score from first test
gs_1.best_score_

0.7950392322890206

In [36]:
#printing best params of first test
gs_1.best_params_

{'cvec__max_features': 4000,
 'cvec__ngram_range': (1, 1),
 'cvec__stop_words': ['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're",
  "you've",
  "you'll",
  "you'd",
  'your',
  'yours',
  'yourself',
  'yourselves',
  'he',
  'him',
  'his',
  'himself',
  'she',
  "she's",
  'her',
  'hers',
  'herself',
  'it',
  "it's",
  'its',
  'itself',
  'they',
  'them',
  'their',
  'theirs',
  'themselves',
  'what',
  'which',
  'who',
  'whom',
  'this',
  'that',
  "that'll",
  'these',
  'those',
  'am',
  'is',
  'are',
  'was',
  'were',
  'be',
  'been',
  'being',
  'have',
  'has',
  'had',
  'having',
  'do',
  'does',
  'did',
  'doing',
  'a',
  'an',
  'the',
  'and',
  'but',
  'if',
  'or',
  'because',
  'as',
  'until',
  'while',
  'of',
  'at',
  'by',
  'for',
  'with',
  'about',
  'against',
  'between',
  'into',
  'through',
  'during',
  'before',
  'after',
  'above',
  'below',
  'to',
  'from',
  'up',
  'down',
  'in

In [41]:
#creating second param dict for next iteration of test for grid search
params_2 = {
    'cvec__max_features':[4000,5000,6000],
    'cvec__stop_words':[stopwords.words('english')],
    'cvec__min_df':[2,3]
}
gs_2 = GridSearchCV(pipe,param_grid=params_2,cv=3)
gs_2.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('svc', SVC())]),
             param_grid={'cvec__max_features': [4000, 5000, 6000],
                         'cvec__min_df': [2, 3],
                         'cvec__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                               'our', 'ours', 'ourselves',
                                               'you', "you're", "you've",
                                               "you'll", "you'd", 'your',
                                               'yours', 'yourself',
                                               'yourselves', 'he', 'him', 'his',
                                               'himself', 'she', "she's", 'her',
                                               'hers', 'herself', 'it', "it's",
                                               'its', 'itself', ...]]})

In [42]:
#best score for second test
gs_2.best_score_

0.7945733019931648

In [43]:
#best params for second test
gs_2.best_params_

{'cvec__max_features': 6000,
 'cvec__min_df': 3,
 'cvec__stop_words': ['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're",
  "you've",
  "you'll",
  "you'd",
  'your',
  'yours',
  'yourself',
  'yourselves',
  'he',
  'him',
  'his',
  'himself',
  'she',
  "she's",
  'her',
  'hers',
  'herself',
  'it',
  "it's",
  'its',
  'itself',
  'they',
  'them',
  'their',
  'theirs',
  'themselves',
  'what',
  'which',
  'who',
  'whom',
  'this',
  'that',
  "that'll",
  'these',
  'those',
  'am',
  'is',
  'are',
  'was',
  'were',
  'be',
  'been',
  'being',
  'have',
  'has',
  'had',
  'having',
  'do',
  'does',
  'did',
  'doing',
  'a',
  'an',
  'the',
  'and',
  'but',
  'if',
  'or',
  'because',
  'as',
  'until',
  'while',
  'of',
  'at',
  'by',
  'for',
  'with',
  'about',
  'against',
  'between',
  'into',
  'through',
  'during',
  'before',
  'after',
  'above',
  'below',
  'to',
  'from',
  'up',
  'down',
  'in',
  'out'

In [ ]:
#creating third param dict for next iteration of testing
params_3 = {
    'cvec__max_features':[7000,9000,11000],
    'cvec__stop_words':[stopwords.words('english')],
    'cvec__min_df':[4,5],
    'cvec__ngram_range':[(1,1),(2,2),(3,3),(4,4)],
    'cvec__analyzer':['word','char_wb']
}
gs_3 = GridSearchCV(pipe,param_grid=params_3,cv=3)
gs_3.fit(X_train,y_train)

In [ ]:
#best score for third test
gs_3.best_score_

In [ ]:
#best params for third test
gs_3.best_params_